In [5]:
import os
import pandas as pd
import glob
import shutil
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from sqlalchemy import create_engine

# Database configuration constants
DBMS = 'postgresql'
DRIVER = 'psycopg2'
USER = 'default'
PASSWORD = 'K1PgwR4caFev'
HOST = 'ep-summer-frog-90198346-pooler.us-east-1.postgres.vercel-storage.com'
PORT = '5432'
DB_NAME = 'verceldb'
# Column mapping

column_mapping = {
    "Row ID":"row_id",
    "Order Priority":"order_prio",
    "Discount":"discount",
    "Unit Price":"unit_price",
    "Shipping Cost":"ship_cost",
    "Customer ID":"customer_id",
    "Customer Name":"customer_name",
    "Ship Mode":"ship_mode",
    "Customer Segment":"customer_segment",
    "Product Category":"product_cat",
    "Product Sub-Category":"product_subcat",
    "Product Container":"product_cont",
    "Product Name":"product_name",
    "Product Base Margin":"product_base_margin",
    "Country":"country",
    "Region":"region",
    "State or Province":"state",
    "City":"city",
    "Postal Code":"postal_code",
    "Order Date":"order_date",
    "Ship Date":"ship_date",
    "Profit":"profit",
    "Quantity ordered new":"qty_ordered",
    "Sales":"sales",
    "Order ID":"order_id"
}	

# Setup database connection
engine = create_engine(f'{DBMS}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}')

class MyHandler(FileSystemEventHandler):
    def process_file(self, file_path):
        df = pd.read_excel(file_path)
        df_mapped = df.rename(columns=column_mapping)
        df_mapped.to_sql('temp', engine, if_exists='append', index=False, method=None)
        
        # Specify the destination directory
        destination_directory = './Data/inserted/'
        destination_file = os.path.join(destination_directory, os.path.basename(file_path))
        
        # Move the file
        shutil.move(file_path, destination_file)
        print(f'Moved {file_path} to {destination_directory}')

    def on_created(self, event):
        if event.is_directory:
            return
        
        if event.src_path.endswith('.xlsx'):
            self.process_file(event.src_path)

if __name__ == "__main__":
    path = './Data/to_insert/'
    event_handler = MyHandler()
    observer = Observer()
    observer.schedule(event_handler, path, recursive=False)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    
    observer.join()
    engine.dispose()


Moved ./Data/to_insert/SuperStoreUS1.1.xlsx to ./Data/inserted/
